#### Reinforcement Learning and Decision Making &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Homework #1

# &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Planning in MDPs

## Description

You are given an $N$-sided die, along with a corresponding Boolean mask
vector, `is_bad_side` (i.e., a vector of ones and zeros). You can assume
that $1<N\leq30$, and the vector `is_bad_side` is also of size $N$ and
$1$ indexed (since there is no $0$ side on the die). The game of DieN is
played as follows:

1.  You start with $0$ dollars.

2.  At any time you have the option to roll the die or to quit the game.

    1.  **ROLL**:

        1.  If you roll a number not in `is_bad_side`, you receive that
            many dollars (e.g., if you roll the number $2$ and $2$ is
            not a bad side -- meaning the second element of the vector
            `is_bad_side` is $0$, then you receive $2$ dollars). Repeat
            step 2.

        2.  If you roll a number in `is_bad_side`, then you lose all the
            money obtained in previous rolls and the game ends.

    2.  **QUIT**:

        1.  You keep all the money gained from previous rolls and the
            game ends.

## Procedure

-   You will implement your solution using the `solve()` method
    in the code below.
    
-   Your return value should be the number of dollars you expect to
    win for a specific value of `is_bad_side`, if you follow an
    optimal policy. That is, what is the value of the optimal
    state-value function for the initial state of the game (starting
    with $0$ dollars)? Your answer must be correct to $3$ decimal
    places, truncated (e.g., 3.14159265 becomes 3.141).

-   To solve this problem, you will need to determine an optimal policy
    for the game of DieN, given a particular configuration of the die.
    As you will see, the action that is optimal will depend on your
    current bankroll (i.e., how much money you've won so far).

-   You can try solving this problem by creating an MDP of the game
    (states, actions, transition function, reward function, and assume a
    discount rate of $\gamma=1$) and then calculating the optimal
    state-value function.

## Resources

The concepts explored in this homework are covered by:

-   Lecture Lesson 1: Smoov & Curly's Bogus Journey

-   Chapter 3 (3.6 Optimal Policies and Optimal Value Functions) and
    Chapter 4 (4.3-4.4 Policy Iteration, Value Iteration) of
    http://incompleteideas.net/book/the-book-2nd.html

-   Chapters 1-2 of 'Algorithms for Sequential Decision Making', M.
    Littman, 1996

## Submission

-   The due date is indicated on the Canvas page for this assignment.
    Make sure you have your timezone in Canvas set to ensure the
    deadline is accurate.

-   Submit your finished notebook on Gradescope. Your grade is based on
    a set of hidden test cases. You will have unlimited submissions -
    only the last score is kept.

-   Use the template below to implement your code. We have also provided
    some test cases for you. If your code passes the given test cases,
    it will run (though possibly not pass all the tests) on Gradescope.

-   Gradescope is using *python 3.6.x* and *numpy==1.18.0*, and you can
    use any core library (i.e., anything in the Python standard library).
    No other library can be used.  Also, make sure the name of your
    notebook matches the name of the provided notebook.  Gradescope times
    out after 10 minutes.

In [ ]:
#################
# DO NOT REMOVE
# Versions
# numpy==1.18.0
################
import numpy as np

class MDPAgent(object):
    def __init__(self):
        self.money_gain = 0
        pass
    
    def expectation(self, is_bad_side, money_gain):
        """Calculate expectation according to die type and current gain"""
        die_size = len(is_bad_side)
        exp_sum = 0
        for i in range(0, die_size):
            if is_bad_side[i] == 1:
                # forefeit all money
                exp_sum = exp_sum - money_gain
            elif is_bad_side[i] == 0:
                # add die number
                exp_sum = exp_sum + i + 1
                
        exp = exp_sum / die_size
        
        return exp
    
    def rollresult(self, is_bad_side, money_gain):
        """the result of this roll"""
        # v value
        die_size = len(is_bad_side)
        result = np.zeros(die_size)
        for i in range(0, die_size):
            if is_bad_side[i] == 1:
                # forefeit all money
                result[i] = - money_gain
            elif is_bad_side[i] == 0:
                # add die number
                result[i] = i + 1
        
        return result

    def solve(self, is_bad_side):
        """Implement the agent"""
        money_gain = self.money_gain
        
        # step 1. determine maximum roll number
        # if expectations are all negative or zero, stop
        
        # initial roll must be able to run unless all sides are bad or all good
        is_bad_side = np.array(is_bad_side)
        
        if (is_bad_side == 1).sum() == len(is_bad_side):
            # all sides are bad
            exp = 0
        elif (is_bad_side == 0).sum() == len(is_bad_side):
            # all sides are good
            exp = 999999999
        else: 
            # search maximum depth 
            ini_exp = self.expectation(is_bad_side, money_gain)
            ini_result = self.rollresult(is_bad_side, money_gain)
            result = self.rollresult(is_bad_side, money_gain)
        
            roll_num = 1
            roll = 1  # roll flag. =1: keep rolling
        
            # step 2
            while roll == 1:
                roll = 0
                result_sub = []
                # for each positive result, calculate its expectation and store
                for i in range(0, len(result)):
                    if result[i] > 0:
                        # un-failed roll, move forawrd a layer
                        if self.expectation(is_bad_side, result[i]) > 0:
                            # if any of the sub branch have positive expectation, keep rolling
                            roll = 1 
                            result_sub.extend(result[i] + self.rollresult(is_bad_side, result[i]))
                            # Update result
                result = result_sub
                            
                # Update roll depth after looped all branches
                if roll == 1:
                    roll_num = roll_num + 1
            
            print(roll_num)
            # known how many times can roll, build all results and find expectation
            if roll_num == 1:
                exp = ini_exp
            else:
                exp = ini_exp
                roll_n = 2
                result_store = ini_result
                result_temp = []
                while roll_n <= roll_num:
                    print('roll n = ', roll_n)
                    exp_sub = []
                    for i in range(0, len(result_store)):
                        if result_store[i] > 0:
                            if self.expectation(is_bad_side, result_store[i]) > 0:
                                # roll die
                                exp_sub.append(self.expectation(is_bad_side, result_store[i])+result_store[i])
                                result_temp.extend(result_store[i] + self.rollresult(is_bad_side, result_store[i]))
                            else:
                                # quit, keep current gain
                                exp_sub.append(result_store[i])
                                result_temp.extend(np.zeros(len(is_bad_side)) + result_store[i]) 
                        else:
                            # this branch already failed
                            exp_sub.append(0)
                            result_temp.extend(np.zeros(len(is_bad_side)))
                            
                    result_store = result_temp
                    exp_sum = 0
                    for i in range(0, len(exp_sub)):
                        exp_sum = exp_sum + exp_sub[i]
                                    
                    exp = (exp_sum) / ((len(is_bad_side))**(roll_n - 1))
                    roll_n = roll_n + 1
                    
                    for i in range(0, len(result_store)):
                        if result_store[i] < 0:
                            result_store[i] = 0
        return exp

    
    

In [ ]:
agent = MDPAgent()
agent.solve(is_bad_side=[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])

## Test cases

We have provided some test cases for you to help verify your implementation.

In [ ]:
## DO NOT MODIFY THIS CODE.  This code will ensure that your submission
## will work proberly with the autograder

import unittest

class TestDieNNotebook(unittest.TestCase):
    def test_case_1(self):
        agent = MDPAgent()
        np.testing.assert_almost_equal(
            agent.solve(is_bad_side=[1, 1, 1, 0, 0, 0]),
            2.583,
            decimal=3
        )
        
    def test_case_2(self):
        agent = MDPAgent()
        np.testing.assert_almost_equal(
            agent.solve(
                is_bad_side=[1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0]
            ),
            7.379,
            decimal=3
        )
        
    def test_case_3(self):
        agent = MDPAgent()

        np.testing.assert_almost_equal(
            agent.solve(
                is_bad_side=[1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0]
            ),
            6.314,
            decimal=3
        )

unittest.main(argv=[''], verbosity=2, exit=False)